In [1]:
# ============================================
# Cell 1: Environment & clients
# ============================================

import os
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

# --- Basic GCP config ---
PROJECT_ID   = "willy-poc"
BQ_LOCATION  = "US"
DATASET      = "chatbot_agent"
TABLE_NAME   = "customer_support_faqs"

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

# --- Init BigQuery client ---
bq = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

# --- Init Vertex AI (same project, us-central1) ---
vertexai.init(project=PROJECT_ID, location="us-central1")

# --- GenAI model for answer generation (Gemini 2.5 Pro) ---
GEN_MODEL = GenerativeModel("gemini-2.5-pro")
GEN_CONFIG = GenerationConfig(
    temperature=0.3,      # low-ish temperature for stable customer support tone
    max_output_tokens=1024,
)

print("Env ready:", PROJECT_ID, DATASET, TABLE_NAME)


Env ready: willy-poc chatbot_agent customer_support_faqs


/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [2]:
# ============================================
# Cell 2: Create / update remote embedding model
# ============================================
# This model will be used by BigQuery ML.GENERATE_EMBEDDING
# to generate vector embeddings for FAQ content.
# We use `gemini-embedding-001` (multilingual-friendly).

# NOTE: Replace CONNECTION with your existing BQ connection name if needed.
CONNECTION = "us.jtcg"  # e.g. us.jtcg / us.chatbot_agent

sql_create_remote_model = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`
REMOTE WITH CONNECTION `{CONNECTION}`
OPTIONS (
  ENDPOINT = 'gemini-embedding-001'
);
"""

bq.query(sql_create_remote_model).result()
print("Remote embedding model ready:", f"{PROJECT_ID}.{DATASET}.text_embedding_model")


Remote embedding model ready: willy-poc.chatbot_agent.text_embedding_model


In [3]:
# ============================================
# Cell 3: Add embedding column & generate embeddings
# ============================================
# Source table schema (already created by your previous pipeline):
# - id (INT64)
# - question_en / answer_en (STRING)
# - question_zh / answer_zh (STRING)
# - lang, source, category (STRING)
#
# Here we:
# 1) Add an ARRAY<FLOAT64> column `answer_embedding` if it doesn't exist.
# 2) Use ML.GENERATE_EMBEDDING over (question_zh + answer_zh) as content.
# 3) Update the table with generated embeddings.

# 1) Add vector column if not exists
sql_add_column = f"""
ALTER TABLE `{PROJECT_ID}.{DATASET}.{TABLE_NAME}`
ADD COLUMN IF NOT EXISTS answer_embedding ARRAY<FLOAT64>;
"""
bq.query(sql_add_column).result()
print("Ensured column answer_embedding exists.")

# 2) Generate embeddings into a temp table
sql_tmp_emb = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.tmp_faq_emb` AS
SELECT
  id,
  ml_generate_embedding_result AS emb
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`,
  (
    SELECT
      id,
      CONCAT(
        COALESCE(question_zh, ''),
        ' ',
        COALESCE(answer_zh, '')
      ) AS content
    FROM `{PROJECT_ID}.{DATASET}.{TABLE_NAME}`
  )
);
"""

bq.query(sql_tmp_emb).result()
print("Temporary embedding table created:", f"{PROJECT_ID}.{DATASET}.tmp_faq_emb")

# 3) Update main table with embeddings
sql_update_emb = f"""
UPDATE `{PROJECT_ID}.{DATASET}.{TABLE_NAME}` f
SET answer_embedding = t.emb
FROM `{PROJECT_ID}.{DATASET}.tmp_faq_emb` t
WHERE f.id = t.id;
"""

bq.query(sql_update_emb).result()
print("Embeddings generated & updated into", f"{PROJECT_ID}.{DATASET}.{TABLE_NAME}")


Ensured column answer_embedding exists.
Temporary embedding table created: willy-poc.chatbot_agent.tmp_faq_emb
Embeddings generated & updated into willy-poc.chatbot_agent.customer_support_faqs


In [4]:
# ============================================
# Cell 4: Vector search helper (FAQ only, Chinese)
# ============================================
# We use BigQuery VECTOR_SEARCH over `answer_embedding`.
# To keep downstream code simple, we alias:
#   question_zh -> title
#   answer_zh   -> answer
#   url         -> NULL (no URL in this dataset)

from google.cloud import bigquery

def search_knowledge(query_text: str, top_k: int = 5):
    """
    Semantic search over FAQ table using `answer_embedding`.
    Returns rows with:
      - id
      - title  (alias of question_zh)
      - answer (alias of answer_zh)
      - url    (currently NULL)
      - distance (smaller = more similar)
    """
    sql = f"""
    SELECT 
        base.id,
        base.question_zh AS title,
        base.answer_zh   AS answer,
        NULL             AS url,
        distance
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET}.{TABLE_NAME}`,
        'answer_embedding',
        (
            SELECT ml_generate_embedding_result AS query_embedding
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`,
                (SELECT @query_text AS content)
            )
        ),
        top_k => {top_k},
        options => '{{"use_brute_force": true}}'
    )
    ORDER BY distance
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("query_text", "STRING", query_text)
        ]
    )
    return bq.query(sql, job_config=job_config).result()

print("Vector search helper ready.")


Vector search helper ready.


In [5]:
# ============================================
# Cell 5: LLM reply layer (Chinese only)
# ============================================
# This function wraps:
#   - user query
#   - retrieved FAQ blocks
# into a single prompt for Gemini 2.5 Pro,
# and returns the final Chinese answer.

def generate_llm_reply(user_query: str, context_blocks: list[str]) -> str:
    """
    Use Gemini 2.5 Pro to generate final Chinese answer for MakTek-like FAQ.
    - user_query: original user question (Chinese)
    - context_blocks: text blocks built from retrieved FAQ entries
    """
    if not context_blocks:
        context_text = "（目前沒有檢索到任何 FAQ 內容。）"
    else:
        context_text = "\n\n".join(
            f"[資料 {i+1}]\n{block}" for i, block in enumerate(context_blocks)
        )

    prompt = f"""
你是一位電商平台（例如 MakTek）的官方客服 AI，請用自然、有禮貌的繁體中文回答使用者的問題。

【使用者問題】
{user_query}

【系統檢索到的相關資料】
{context_text}

你的任務如下：

1. 仔細閱讀以上所有「檢索到的資料」，確認其中是否已經包含對使用者問題的「明確且具體的答案」，例如：
   - 如何註冊或登入帳號
   - 支援哪些付款方式
   - 如何查詢訂單進度
   - 退貨／退款的條件與流程
   - 是否可以取消訂單、如何操作
   - 運送方式、運費計算、預估到貨時間 …… 等等。

2. 如果你能明確判斷某一筆資料（例如 [資料 1] 或 [資料 2]）最貼近使用者問題，
   請「優先以那一筆資料為主」整理答案，不需要平均參考所有資料，
   以避免產生模糊或矛盾的說明。

3. 只要有「任一筆」資料已經直接回應了使用者的提問，就請以那筆資料為主，
   幫使用者整理成一段「具體而且有自信的回答」：
   - 你可以直接引用或適度改寫該段文字；
   - 在這種情況下，不要說「資料不足」或「無法確認」。

4. 只有在「所有」檢索到的資料：
   - 和問題主題明顯無關，或
   - 完全沒有提供任何有用線索
   時，你才可以回答：
   「目前無法根據現有資料做出可靠判斷」，並建議使用者改由真人客服協助。

5. 對於退貨、退款、訂單處理、付款方式、運送與運費等「政策與條款」：
   - 你只能根據上述【系統檢索到的相關資料】內容作答；
   - 不可以自行臆測、延伸或發明新的規則。

6. 回覆時請：
   - 先用一兩句話清楚給出「主要答案」；
   - 再簡短補充 1–2 個重要注意事項或下一步建議（如果有）；
   - 語氣保持專業、可信、友善，避免太冗長。

最後，請只輸出要給使用者看的「最終回覆內容」：
- 不要重複逐條列出【系統檢索到的相關資料】原文，
- 也不要解釋你的思考過程或使用到哪些規則。
"""
    try:
        response = GEN_MODEL.generate_content(
            prompt,
            generation_config=GEN_CONFIG,
        )
        text = (response.text or "").strip()
        if text:
            return text
    except Exception as e:
        print("LLM error in generate_llm_reply:", e)

    # Fallback: safe conservative message in Chinese
    return "抱歉，產生回答時發生問題，建議您稍後再試，或直接改由真人客服協助。"

print("LLM reply layer ready.")


LLM reply layer ready.


In [6]:
# ============================================
# Cell 6: FAQ handler & main entry (answer_with_rag)
# ============================================
# This is a pure FAQ RAG pipeline:
#   user query -> vector search -> (optional keyword check) -> LLM or direct answer.

def handle_faq(query: str, top_k: int = 5) -> str:
    """
    FAQ flow:
    1) Use BigQuery VECTOR_SEARCH to get top_k FAQs.
    2) If the best hit is clearly on the same topic (via simple keyword check),
       respond directly with that FAQ answer.
    3) Otherwise, send multiple FAQ entries to LLM and let it consolidate.
    """
    query = (query or "").strip()
    if not query:
        return ""

    rows = list(search_knowledge(query, top_k=top_k))

    # No FAQ at all -> tell LLM "no context", let it respond conservatively
    if not rows:
        return generate_llm_reply(query, [])

    best = rows[0]
    distance = getattr(best, "distance", None)
    title    = (best.title  or "").strip()
    answer   = (best.answer or "").strip()
    url      = (getattr(best, "url", None) or "").strip()  # currently always empty

    # Simple keyword alignment for this MakTek-like dataset (Chinese FAQ)
    KEYWORDS = [
        # account / login
        "帳號", "註冊", "登入", "密碼",
        # payment
        "付款", "付款方式", "信用卡", "金融卡", "PayPal", "支付",
        # order / shipping
        "訂單", "出貨", "運送", "配送", "運費", "到貨", "追蹤",
        # return / refund / cancel
        "退貨", "退款", "取消訂單", "取消", "退貨政策",
        # discount / coupon
        "折扣", "優惠碼", "優惠券",
    ]

    text_for_match = title + answer
    has_keyword_match = any((kw in query) and (kw in text_for_match) for kw in KEYWORDS)

    # If semantic distance is decent + keyword aligned -> direct FAQ answer
    if (distance is None or distance < 0.8) and has_keyword_match and answer:
        parts = [answer]
        # If we had URL(s), we could append them here.
        # if url:
        #     parts.append(f"\n\n更多詳細說明，您可以參考：{url}")
        return "".join(parts).strip()

    # Otherwise, build multiple context blocks and send to LLM.
    context_blocks = []
    for r in rows:
        t_title    = (r.title  or "").strip()
        t_answer   = (r.answer or "").strip()
        t_url      = (getattr(r, "url", None) or "").strip()
        t_distance = getattr(r, "distance", None)

        block_lines = []
        if t_title:
            block_lines.append(f"FAQ 問題：{t_title}")
        if t_answer:
            block_lines.append(f"FAQ 答案：{t_answer}")
        if t_url:
            block_lines.append(f"相關連結：{t_url}")
        if t_distance is not None:
            block_lines.append(f"語意距離（數值越小越相近）：{t_distance}")

        context_blocks.append("\n".join(block_lines))

    return generate_llm_reply(query, context_blocks)


def answer_with_rag(query_text: str, top_k: int = 5) -> str:
    """
    Main entry:
    - For this simplified version, we only handle FAQ-style questions.
    - Future extensions (product recommendation / order lookup) can be added
      as separate handlers and dispatched here based on intent.
    """
    query_text = (query_text or "").strip()
    if not query_text:
        return ""
    return handle_faq(query_text, top_k=top_k)

print("FAQ handler & main entry ready (FAQ-only RAG).")


FAQ handler & main entry ready (FAQ-only RAG).


In [7]:
# ============================================
# Cell 7: Simple test cases (Chinese)
# ============================================
# These examples are aligned with the translated MakTek FAQ dataset.
# They help validate:
#   - vector search quality
#   - direct-answer vs LLM consolidation behavior

test_queries = [
    "我要怎麼註冊帳號？",
    "你們接受哪些付款方式？",
    "我要怎麼查詢我的訂單進度？",
    "你們的退貨政策是什麼？",
    "已經下單了可以取消嗎？",
    "請問運送方式跟運費如何計算？",
]

for q in test_queries:
    print("==================================================")
    print("Q:", q)

    # Show top-3 vector search hits (for debugging / article screenshots)
    rows = list(search_knowledge(q, top_k=3))
    print("\n[Vector search top-3]")
    for i, r in enumerate(rows, start=1):
        print(f"- Hit #{i}: id={r.id}, distance={r.distance}")
        print(f"  問題：{(r.title or '').strip()}")
        print(f"  答案：{(r.answer or '')[:60].strip()}...")
    if not rows:
        print("  (no hits)")

    # Show final RAG answer
    print("\n[Agent 回覆]")
    ans = answer_with_rag(q, top_k=5)
    print(ans)
    print()


Q: 我要怎麼註冊帳號？

[Vector search top-3]
- Hit #1: id=1, distance=0.4461806710763566
  問題：請問要怎麼註冊帳號？
  答案：若要建立帳號，請點選我們網站右上角的「註冊」按鈕，並依照指示完成註冊程序。...
- Hit #2: id=17, distance=0.7218236765305155
  問題：請問可以免註冊帳號直接訂購嗎？
  答案：是的，您不需要註冊 account，也可以直接以訪客身份下單喔。不過，若您註冊 account，將能享有更多便利，例如方...
- Hit #3: id=161, distance=0.7383065185212451
  問題：請問我該如何更新我的帳號資訊？
  答案：如需更新您的會員帳號資訊，請登入您的會員帳號，點選「帳戶設定」頁面，並完成相關修改即可。...

[Agent 回覆]
若要建立帳號，請點選我們網站右上角的「註冊」按鈕，並依照指示完成註冊程序。

Q: 你們接受哪些付款方式？

[Vector search top-3]
- Hit #1: id=2, distance=0.6041801693785498
  問題：請問貴公司接受哪些付款方式呢？
  答案：我們接受主要信用卡、金融卡以及 PayPal 作為線上訂單的付款方式。...
- Hit #2: id=14, distance=0.8253338403015604
  問題：請問我的個人資料和付款資訊有保障嗎？
  答案：好的，我們非常重視您的個人資料與付款資訊的安全性。我們採用業界標準的加密技術，並遵循嚴格的安全協定，以確保您的資訊受到完...
- Hit #3: id=13, distance=0.8320609226623199
  問題：請問可以電話訂購嗎？
  答案：不好意思，我們目前沒有提供電話訂購服務喔。建議您直接透過我們的網站下單，這樣交易過程會更順暢、更安全喔！...

[Agent 回覆]
我們接受主要信用卡、金融卡以及 PayPal 作為線上訂單的付款方式。

Q: 我要怎麼查詢我的訂單進度？

[Vector search top-3]
- Hit #1: id=3, distance=0.4653280161344369
  問題：請問我該如何查